In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.utils import resample
from scipy.sparse import vstack
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

import numpy as np
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterSampler
from sklearn.utils import resample
from tqdm import tqdm
from joblib import Parallel, delayed
from scipy.sparse import vstack

from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler
from kneed import KneeLocator
import matplotlib.pyplot as plt

from sklearn.decomposition import TruncatedSVD

from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
from itertools import product

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.manifold import TSNE
import numpy as np

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import matplotlib.patches as mpatches
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


##### EDA

In [ ]:
# Adjust the path to where your dataset is stored
dataset = pd.read_csv('dataset.csv')

In [ ]:
dataset.info()

In [ ]:
dataset.head()

In [ ]:
distinct_games = dataset['app_name'].drop_duplicates()
selected_games = distinct_games.sample(10, random_state=42) 

sample_rows = pd.DataFrame()

for game in selected_games:
    sample_rows = sample_rows.append(dataset[dataset['app_name'] == game].sample(1, random_state=42))

print(sample_rows)


In [ ]:
# Set display options
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)

# Randomly select 10 different games
distinct_games = dataset['app_name'].drop_duplicates()
selected_games = distinct_games.sample(10, random_state=42) 

# Get one review from each of the selected games
sample_rows = pd.DataFrame()

for game in selected_games:
    sample_rows = sample_rows.append(dataset[dataset['app_name'] == game].sample(1, random_state=42))

# Print the selected rows
print(sample_rows)


In [ ]:
dataset.isnull().sum()

In [ ]:
# Histogram for 'review_score' column
plt.figure(figsize=(10,6))
sns.histplot(data=dataset, x='review_score', kde=True)
plt.title('Distribution of Review Scores')
plt.show()

# Bar chart for 'review_votes' column
plt.figure(figsize=(10,6))
sns.countplot(x='review_votes', data=dataset)
plt.title('Distribution of Review Votes')
plt.show()

In [ ]:
# Get frequency counts of 'review_score' column
review_score_counts = dataset['review_score'].value_counts()
print("Frequency counts of review scores:")
print(review_score_counts)

# Get frequency counts of 'review_votes' column
review_votes_counts = dataset['review_votes'].value_counts()
print("\nFrequency counts of review votes:")
print(review_votes_counts)


In [ ]:
print(dataset['app_name'].value_counts())

### Cleaning

In [ ]:
# Drop the rows with missing 'app_name' or 'review_text'
dataset.dropna(subset=['app_name', 'review_text'], inplace=True)

# First, create a separate dataframe with games having more 1s than 0s in review_votes.
games_with_more_1s = dataset[dataset['review_votes'] == 1].groupby('app_id').filter(lambda x: x['review_votes'].sum() > len(x) / 2)

# Now create a balanced dataset with equal number of positive and negative reviews
positive_reviews = games_with_more_1s[games_with_more_1s['review_score'] == 1]
negative_reviews = games_with_more_1s[games_with_more_1s['review_score'] == -1]

# Find the minimum size between positive and negative reviews
min_size = min(len(positive_reviews), len(negative_reviews))

# Sample from the larger dataset to match the size of the smaller dataset
positive_reviews_balanced = positive_reviews.sample(min_size)
negative_reviews_balanced = negative_reviews.sample(min_size)

# Concatenate the positive and negative reviews to form a balanced dataset
balanced_reviews = pd.concat([positive_reviews_balanced, negative_reviews_balanced])

# Shuffle the dataset
balanced_reviews = balanced_reviews.sample(frac=1).reset_index(drop=True)

# Show the distribution of review_scores in the balanced dataset
print(balanced_reviews['review_score'].value_counts())

In [ ]:
# Define the number of samples
n_samples = 50000 

# Sample from the balanced positive and negative reviews
positive_reviews_sampled = positive_reviews_balanced.sample(n_samples)
negative_reviews_sampled = negative_reviews_balanced.sample(n_samples)

# Concatenate the positive and negative reviews to form a balanced and reduced dataset
balanced_reviews_reduced = pd.concat([positive_reviews_sampled, negative_reviews_sampled])

# Shuffle the dataset
balanced_reviews_reduced = balanced_reviews_reduced.sample(frac=1).reset_index(drop=True)

# Show the distribution of review_scores in the reduced dataset
print(balanced_reviews_reduced['review_score'].value_counts())

In [ ]:
# Review the structure of the balanced dataset
print("\nStructure of the dataset:")
print(balanced_reviews_reduced.info())

# Check for missing values
print("\nMissing values in each column:")
print(balanced_reviews_reduced.isnull().sum())

# Show the number of unique games in the balanced dataset
print("\nNumber of unique games: ", balanced_reviews_reduced['app_id'].nunique())

# Show the number of unique reviews in the balanced dataset
print("\nNumber of unique reviews: ", balanced_reviews_reduced['review_text'].nunique())

# Show the distribution of review_scores
print("\nDistribution of review scores: ")
print(balanced_reviews_reduced['review_score'].value_counts())

# Show the distribution of review_votes
print("\nDistribution of review votes: ")
print(balanced_reviews_reduced['review_votes'].value_counts())


In [ ]:
# Remove rows where 'review_text' contains 'early access reviews'
dataset= dataset[~dataset['review_text'].str.contains('early access reviews')]
# Remove rows where 'review_text' contains 'early access reviews'
dataset = dataset[~dataset['review_text'].str.contains('early access review')]

In [ ]:
# Save the cleaned dataset to a new CSV file
balanced_reviews_reduced.to_csv('cleaned_dataset.csv', index=False)

### pre-processing

In [ ]:
# Read CSV file
df = pd.read_csv('cleaned_dataset.csv')

In [ ]:
def clean_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)  # Keep only alphabets
    text = text.lower()  # Convert to lowercase
    return text

# Perform text cleaning
df['review_text'] = df['review_text'].apply(clean_text)


In [ ]:
df['review_text'] = df['review_text'].apply(nltk.word_tokenize)

In [ ]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return [word for word in text if word not in stop_words]

df['review_text'] = df['review_text'].apply(remove_stopwords)

In [ ]:
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    return [lemmatizer.lemmatize(word) for word in text]

df['review_text'] = df['review_text'].apply(word_lemmatizer)

In [ ]:
def join_text(text):
    return ' '.join(text)

df['review_text'] = df['review_text'].apply(join_text)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['review_text'])

In [ ]:
# Save the cleaned dataset to a new CSV file
df.to_csv('preprocessedCleanedData.csv', index=False)

### Sentiment Analysis Models

In [ ]:
# Load the data
df = pd.read_csv('preprocessedCleanedData.csv')

# Split the data into training and testing sets using stratified sampling
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

# Split the remaining data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.15, random_state=42, stratify=y_temp)

In [ ]:
# Define a Naive Bayes model
mnb = MultinomialNB()

# Define a hyperparameter grid to search over
param_grid = {
    'alpha': [0.1, 0.5, 1, 5, 10],
    'fit_prior': [True, False],
    # Assuming you have 2 classes, change [0.5, 0.5] accordingly if you have a different number of classes
    'class_prior': [[0.1, 0.9], [0.2, 0.8], [0.3, 0.7], [0.4, 0.6], [0.5, 0.5], [0.6, 0.4], [0.7, 0.3], [0.8, 0.2], [0.9, 0.1], None]
}

# Set up the grid search
grid_search = GridSearchCV(mnb, param_grid, cv=5)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(best_params)

# Train a new MultinomialNB with the best parameters
mnb_best = MultinomialNB(**best_params)
mnb_best.fit(X_train, y_train)

In [ ]:
# Validate the model using validation set
y_val_pred = mnb_best.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f'Validation Accuracy: {val_accuracy}')

In [ ]:
# Additional metrics like precision, recall and f1-score
print('\nValidation Classification Report:')
print(classification_report(y_val, y_val_pred))

In [ ]:
# Define a SVM model
svc = svm.SVC()

# Define a hyperparameter grid to search over
svm_params = {
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
    "C": [0.1, 1, 10],
    "gamma": ['scale', 'auto'] + list(np.logspace(-4, 2, 10)),
    "degree": [2, 3, 4],
    "coef0": [0, 1, 2],
    "shrinking": [True, False],
    "probability": [True],
    "tol": [1e-4, 1e-3, 1e-2],
    "class_weight": [None, "balanced"]
}

# Get a balanced sample of the training data
sample_size = int(X_train.shape[0] * 0.5)
X_train_sample_neg = resample(X_train[y_train == -1], n_samples=sample_size // 2, replace=False)
X_train_sample_pos = resample(X_train[y_train == 1], n_samples=sample_size // 2, replace=False)
X_train_sample = vstack([X_train_sample_neg, X_train_sample_pos])
y_train_sample = np.concatenate([-np.ones(sample_size // 2), np.ones(sample_size // 2)])

# Implement the progress bar function
def progress_bar_cross_val_score(estimator, X, y, cv):
    scores = []
    for train_index, test_index in tqdm(cv.split(X, y), desc="CV", leave=False):
        X_train_fold, X_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        estimator.fit(X_train_fold, y_train_fold)
        y_pred = estimator.predict(X_test_fold)
        score = f1_score(y_test_fold, y_pred)
        scores.append(score)
    return scores

# Define a function to evaluate a set of parameters
def evaluate_params(params):
    svc.set_params(**params)
    scores = progress_bar_cross_val_score(svc, X_train_sample, y_train_sample, cv=kf)
    mean_score = np.mean(scores)
    return mean_score, params

# Perform the parameter search using KFold and progress bar
kf = KFold(n_splits=5, shuffle=True, random_state=42)
param_list = list(ParameterSampler(svm_params, n_iter=30, random_state=42))

best_score_params = Parallel(n_jobs=-1)(
    delayed(evaluate_params)(params) for params in tqdm(param_list, desc="Parameters"))

best_score, best_params = max(best_score_params, key=lambda x: x[0])

# Print the best parameters
print(f"Best parameters: {best_params}")

# Train the SVM with the best parameters on the entire training set
best_svm = svm.SVC(**best_params)
best_svm.fit(X_train, y_train)

In [ ]:
# Validate the model using validation set
y_val_pred = best_svm.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f'Validation Accuracy: {val_accuracy}')

In [ ]:
# Additional metrics like precision, recall and f1-score
print('\nValidation Classification Report:')
print(classification_report(y_val, y_val_pred))

In [ ]:
# Test the model using testing set
y_test_pred = mnb_best.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'Test Accuracy: {test_accuracy}')

In [ ]:
print('\nTest Classification Report:')
print(classification_report(y_test, y_test_pred))

In [ ]:
# Test the model using test set
y_test_pred = best_svm.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'Test Accuracy: {test_accuracy}')

In [ ]:
# Generate a classification report
print('Test Classification Report:')
print(classification_report(y_test, y_test_pred))

In [ ]:
# Test the SVM model using test set
y_test_pred_svm = best_svm.predict(X_test)
test_accuracy_svm = accuracy_score(y_test, y_test_pred_svm)
print(f'SVM Test Accuracy: {test_accuracy_svm}')

# Generate confusion matrix for SVM
conf_mat_svm = confusion_matrix(y_test, y_test_pred_svm)

# Plot confusion matrix for SVM as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat_svm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.title('SVM Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Test the MNB model using test set
y_test_pred_mnb = mnb_best.predict(X_test)
test_accuracy_mnb = accuracy_score(y_test, y_test_pred_mnb)
print(f'MNB Test Accuracy: {test_accuracy_mnb}')

# Generate confusion matrix for MNB
conf_mat_mnb = confusion_matrix(y_test, y_test_pred_mnb)

# Plot confusion matrix for MNB as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat_mnb, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.title('Naive Bayes Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# Generate prediction probabilities
y_test_proba_svm = best_svm.predict_proba(X_test)[:, 1]
y_test_proba_mnb = mnb_best.predict_proba(X_test)[:, 1]

# Compute ROC curve for SVM
fpr_svm, tpr_svm, _ = roc_curve(y_test, y_test_proba_svm)
roc_auc_svm = auc(fpr_svm, tpr_svm)

# Compute ROC curve for MNB
fpr_mnb, tpr_mnb, _ = roc_curve(y_test, y_test_proba_mnb)
roc_auc_mnb = auc(fpr_mnb, tpr_mnb)

# Plot ROC curves
plt.figure(figsize=(8, 6))
plt.plot(fpr_svm, tpr_svm, label='SVM (area = %0.2f)' % roc_auc_svm)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for SVM')
plt.legend(loc="lower right")
plt.show()

# Plot ROC curves
plt.figure(figsize=(8, 6))
plt.plot(fpr_mnb, tpr_mnb, label='MNB (area = %0.2f)' % roc_auc_mnb)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Naive Bayes')
plt.legend(loc="lower right")
plt.show()


### Clustering Algorithms

In [ ]:
# Use trained models to predict sentiment labels on test set
y_val_pred_mnb = mnb_best.predict(X_val)
y_val_pred_svm = best_svm.predict(X_val)

# Add these labels to your test dataframe
df_test = df.loc[df.index.isin(test_indices)].copy()
df_test['mnb_sentiment'] = y_val_pred_mnb
df_test['svm_sentiment'] = y_val_pred_svm

# Print the number of instances classified as negative and positive by each model
print("SVM: Number of negative instances =", sum(df_test['svm_sentiment'] == -1))
print("SVM: Number of positive instances =", sum(df_test['svm_sentiment'] == 1))
print("MNB: Number of negative instances =", sum(df_test['mnb_sentiment'] == -1))
print("MNB: Number of positive instances =", sum(df_test['mnb_sentiment'] == 1))

In [ ]:
# Add sentiment labels to the test set
df_val = df.loc[test_indices].copy()
df_val['mnb_sentiment'] = y_val_pred_mnb
df_val['svm_sentiment'] = y_val_pred_svm

# Split test data into 4 datasets: SVM Positive, SVM Negative, MNB Positive, MNB Negative
datasets = [df_val[df_val['svm_sentiment'] == label] for label in [-1, 1]] + \
           [df_val[df_val['mnb_sentiment'] == label] for label in [-1, 1]]
dataset_names = ['SVM Negative', 'SVM Positive', 'Naive Bayes Negative', 'Naive Bayes Positive']

In [ ]:
# Prepare to record the best number of clusters for each dataset
best_ks = []

In [ ]:
# Feature extraction
df_val['review_length'] = df_val['review'].apply(lambda x: len(x))
df_val['word_count'] = df_val['review'].apply(lambda x: len(x.split()))

# Feature normalization
scaler = MinMaxScaler()
df_val[['review_length', 'word_count']] = scaler.fit_transform(df_val[['review_length', 'word_count']])

In [ ]:
# Loop over datasets
for df_temp, name in zip(datasets, dataset_names):
    # Check if the dataframe is empty
    if df_temp.empty:
        print(f"No instances of {name} in the test set.")
        best_ks.append(None)
        continue    
    # Get the group's data
    group_data = df_val.loc[df_temp.index, ['review_length', 'word_count']]

    # Use the elbow method to find the best number of clusters
    distortions = []
    K = range(1,12)
    for k in K:
        kmeanModel = KMeans(n_clusters=k)
        kmeanModel.fit(group_data)
        distortions.append(kmeanModel.inertia_)

    # Plot the elbow
    plt.figure(figsize=(15,8))
    plt.plot(K, distortions, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.title('The Elbow Method showing the optimal k for ' + name)
    plt.show()

    # Assuming the "elbow" is located at the point where the plot bends most sharply,
    # we can determine this by finding the point with the maximum curvature.
    kneedle = KneeLocator(K, distortions, curve='convex', direction='decreasing')
    print(f'The optimal number of clusters for {name} is {kneedle.knee}')
    best_ks.append(kneedle.knee)

In [ ]:
# Create a mapping of original indices to X_test indices
idx_mapping = {idx: i for i, idx in enumerate(test_indices)}

# Now we perform k-means clustering for each group using 3 clusters
clusterings = {}
dataset_names = ['SVM Negative', 'SVM Positive', 'Naive Bayes Negative', 'Naive Bayes Positive']

# Initialize TruncatedSVD
svd = TruncatedSVD(n_components=100, random_state=42)

for name in dataset_names:
    # Only proceed if there is data for this group
    if not datasets[dataset_names.index(name)].empty:
        # Get the indices of the reviews in this group
        group_indices = datasets[dataset_names.index(name)].index
        # Map original indices to X_test indices
        mapped_indices = [idx_mapping[idx] for idx in group_indices]
        # Get the corresponding TF-IDF vectors
        group_data = X_val[mapped_indices]
        # Perform dimensionality reduction
        group_data_reduced = svd.fit_transform(group_data)
        # Perform k-means clustering
        kmeans = KMeans(n_clusters=3, random_state=42)
        kmeans.fit(group_data_reduced)
        clusterings[name] = kmeans.labels_
        silhouette_avg = silhouette_score(group_data_reduced, kmeans.labels_)
        print(f"For {name}, the average silhouette score is : {silhouette_avg}")

# Assign the clusters to the original test set
for name, clustering in clusterings.items():
    df_val.loc[datasets[dataset_names.index(name)].index, name + '_cluster'] = clustering

print(df_val.head())

In [ ]:
# Define the parameter space
eps_values = [0.1, 0.5, 1.0, 1.5, 2.0]  
min_samples_values = [5, 10, 15, 20]  

best_params = {}
for name in dataset_names:
    if not datasets[dataset_names.index(name)].empty:
        group_indices = datasets[dataset_names.index(name)].index
        mapped_indices = [idx_mapping[idx] for idx in group_indices]
        group_data = X_test[mapped_indices]
        group_data_reduced = svd.fit_transform(group_data)

        # Initialize the best score for this group
        best_score = -1
        # Iterate over all combinations of eps_values and min_samples_values
        for eps, min_samples in product(eps_values, min_samples_values):
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(group_data_reduced)
            
            # Ignore the -1 labels as they are considered noise by DBSCAN
            core_samples_mask = labels != -1
            labels_core = labels[core_samples_mask]
            group_data_core = group_data_reduced[core_samples_mask]
            
            # Compute the silhouette score
            if len(set(labels_core)) > 1:  # to avoid ValueError
                score = silhouette_score(group_data_core, labels_core)
                if score > best_score:
                    best_score = score
                    best_params[name] = {'eps': eps, 'min_samples': min_samples}

# Print the best parameters
for name, params in best_params.items():
    print(name, params)


In [ ]:
# Define the parameter space
eps_values = [0.1, 0.5, 1.0, 1.5, 2.0]  
min_samples_values = [5, 10, 15, 20]  

best_params = {}
for name in dataset_names:
    if not datasets[dataset_names.index(name)].empty:
        group_indices = datasets[dataset_names.index(name)].index
        mapped_indices = [idx_mapping[idx] for idx in group_indices]
        group_data = X_test[mapped_indices]
        group_data_reduced = svd.fit_transform(group_data)

        # Initialize the best score for this group
        best_score = -1
        # Iterate over all combinations of eps_values and min_samples_values
        for eps, min_samples in product(eps_values, min_samples_values):
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(group_data_reduced)
            
            # Ignore the -1 labels as they are considered noise by DBSCAN
            core_samples_mask = labels != -1
            labels_core = labels[core_samples_mask]
            group_data_core = group_data_reduced[core_samples_mask]
            
            # Compute the silhouette score
            if len(set(labels_core)) > 1:  # to avoid ValueError
                score = silhouette_score(group_data_core, labels_core)
                if score > best_score:
                    best_score = score
                    best_params[name] = {'eps': eps, 'min_samples': min_samples}

# Print the best parameters
for name, params in best_params.items():
    print(name, params)


In [ ]:
# Initialize CountVectorizer
vec = CountVectorizer(stop_words='english', max_features=10)

# Loop over each group and each cluster within the group
for name in dataset_names:
    for cluster in range(3):
        # Get the reviews in this cluster
        reviews = df_val[(df_val[name + '_cluster'] == cluster)]['review_text']
        if reviews.empty:
            continue
        # Get the most common words in these reviews
        word_counts = Counter()
        for _, review in reviews.iteritems():
            words = review.split()
            word_counts.update(words)
        # Print the 10 most common words
        print(f'The 10 most common words in cluster {cluster} of {name} are:')
        for word, count in word_counts.most_common(10):
            print(f'{word}: {count}')
        print('\n')

In [ ]:
# For each sentiment model and each cluster, calculate the average sentiment score
for name in dataset_names:
    for cluster in range(3):
        avg_score = df_val[df_val[name + '_cluster'] == cluster]['review_score'].mean()
        print(f"Average sentiment score for cluster {cluster} in {name}: {avg_score:.2f}")

In [ ]:
# Define the best number of clusters for each sentiment of each model
best_ks = [3, 8, 7, 7]

# Your loop where you analyze each dataset
for i, (dataset, name) in enumerate(zip(datasets, dataset_names)):
    print(f'\n{name}:')

    # Create a copy of the dataset to avoid SettingWithCopyWarning
    dataset = dataset.copy()

    # Transform the text to vectors
    X = vectorizer.transform(dataset['review_text'])
    X_svd = svd.fit_transform(X)

    num_clusters_kmeans = best_ks[i]
    num_clusters_dbscan = len(np.unique(dbscan.labels_))
    
    if num_clusters_kmeans:
        # Perform KMeans clustering with n_init set explicitly to avoid FutureWarning
        kmeans = KMeans(n_clusters=num_clusters_kmeans, n_init=10, random_state=42).fit(X_svd)
        dataset['kmeans_cluster'] = kmeans.labels_
        print(f"Number of clusters created by KMeans: {len(np.unique(kmeans.labels_))}")

    # Perform DBSCAN clustering
    eps = distances[-1] / 2
    min_samples = 2 * X_svd.shape[1]
    dbscan = DBSCAN(eps=eps, min_samples=min_samples).fit(X_svd)
    dataset['dbscan_cluster'] = dbscan.labels_
    print(f"Number of clusters created by DBSCAN: {num_clusters_dbscan}")

    # Calculate average sentiment for each cluster
    if num_clusters_kmeans:
        kmeans_avg_sentiment_svm = dataset.groupby('kmeans_cluster')['svm_sentiment'].mean()
        kmeans_avg_sentiment_mnb = dataset.groupby('kmeans_cluster')['mnb_sentiment'].mean()
        print("Average SVM sentiment for each KMeans cluster:")
        print(kmeans_avg_sentiment_svm)
        print("Average MNB sentiment for each KMeans cluster:")
        print(kmeans_avg_sentiment_mnb)

    dbscan_avg_sentiment_svm = dataset.groupby('dbscan_cluster')['svm_sentiment'].mean()
    dbscan_avg_sentiment_mnb = dataset.groupby('dbscan_cluster')['mnb_sentiment'].mean()
    print("Average SVM sentiment for each DBSCAN cluster:")
    print(dbscan_avg_sentiment_svm)
    print("Average MNB sentiment for each DBSCAN cluster:")
    print(dbscan_avg_sentiment_mnb)

    # PCA and plot
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(X_svd)

    if num_clusters_kmeans:
        cmap = plt.get_cmap('gist_rainbow')
        legend_patches = [mpatches.Patch(color=cmap(i/num_clusters_kmeans), label=f'Cluster {i}') for i in range(num_clusters_kmeans)]
        plt.legend(handles=legend_patches)
        plt.scatter(principalComponents[:, 0], principalComponents[:, 1], c=dataset['kmeans_cluster'], cmap=cmap)
        plt.title('KMeans Clusters')
        plt.show()

    cmap_dbscan = plt.get_cmap('viridis')
    legend_patches_dbscan = [mpatches.Patch(color=cmap_dbscan(i/num_clusters_dbscan), label=(f'Cluster {i}' if i >= 0 else 'Noise')) for i in np.unique(dbscan.labels_)]
    plt.legend(handles=legend_patches_dbscan)
    plt.scatter(principalComponents[:, 0], principalComponents[:, 1], c=dataset['dbscan_cluster'], cmap=cmap_dbscan)
    plt.title('DBSCAN Clusters')
    plt.show()
